# 1. Spark gyakorlat

Spark inicializáció

In [2]:
from pyspark import SparkConf, SparkContext 
import findspark
findspark.init()
findspark.find()
conf = SparkConf().set("spark.local.dir", "/home/jxn/.sparktmp")
sc = SparkContext(conf=conf)

23/11/07 19:55:48 WARN Utils: Your hostname, jxn-linux resolves to a loopback address: 127.0.1.1; using 192.168.1.7 instead (on interface eno1)
23/11/07 19:55:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/07 19:55:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/07 19:55:49 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/11/07 19:55:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


RDD = resilient distributed dataset

Az RDD objektumok immutábilis elosztott kollekciója

RDD létrehozható bármilyen kollekcióból az `sc.parallelize()` függvénnyel.

Az `sc.parallelize()` második argumentumában megadhatjuk hány partíción legyenek elosztva az adatok

In [3]:
tomb = ['a', 'b', 'c', 'd', 'e', 'f']

rdd = sc.parallelize(tomb, 5)
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

Az alábbi függvény visszaadja az RDD partícióinak a számát.

In [3]:
rdd.getNumPartitions()

5

Az RDD-ken alkalmazhatunk transzformációkat és akciókat

A transzformáció RDD-ből új RDD-t hoz létre

Az akciókkat RDD-re alkalmazzuk és egyszerű értékkel térnek vissza

Az alábbi kód létrehoz egy RDD-t, amely 1000 számot tartalmaz

A `filter()` transzformáció szűr a hárommal osztható számokra

A `count()` akicó megszámolja az RDD-ben található értékeket


In [4]:

szamok = sc.parallelize(range(1000)).filter(lambda x: x % 3 == 0)
szamok.count()

334

Transzformációk: `filter`, `map`, `flatMap`, `sort`, `sample`, `union`, `intersection`, `distinct`

Akciók: `count`, `take`, `collect`, `max`, `min`, `reduce`, `foreach`

A nevük elárulja mit csinálnak, de érdemes kipróbálni őket!

Példa: számok összeadása

A `reduce()` akció aggregálja az elemeket egy függvénnyel, amely két argumentumot kap és egy értékkel tér vissza 

Kommutatív és asszociatív függvénynek kell lennie, hogy párhuzamosan végrehajtható legyen


In [6]:
szamok = sc.parallelize(range(10))\
.filter(lambda x: x)\
.reduce(lambda a, b: a + b)

szamok = sc.parallelize(range(10)).sum()

print(szamok)

45


Lemezen lévő fájlból is készíthetünk RDD-t

Az alábbi kód kírja a szöveg első két sorát:

In [6]:
lines = sc.textFile('alkotmany.txt').take(2)

print(lines)

['Mi, a magyar nemzet tagjai, az új évezred kezdetén, felelősséggel minden magyarért, kinyilvánítjuk az alábbiakat:', 'Büszkék vagyunk arra, hogy Szent István királyunk ezer évvel ezelőtt szilárd alapokra helyezte a magyar államot, és hazánkat a keresztény Európa részévé tette.']


Feladat: Grep külön függvénnyel

In [7]:
def filter_row(line):
  grepWord = 'magyar'
  if grepWord in line:
    return True
  else:
    return False


lines = sc.textFile('alkotmany.txt')\
.filter(filter_row)

lines.take(2)

['Mi, a magyar nemzet tagjai, az új évezred kezdetén, felelősséggel minden magyarért, kinyilvánítjuk az alábbiakat:',
 'Büszkék vagyunk arra, hogy Szent István királyunk ezer évvel ezelőtt szilárd alapokra helyezte a magyar államot, és hazánkat a keresztény Európa részévé tette.']

Feladat: Grep

In [8]:
grepWord = 'magyar'

lines = sc.textFile('alkotmany.txt')\
.filter(lambda line: grepWord in line)

lines.take(2)

['Mi, a magyar nemzet tagjai, az új évezred kezdetén, felelősséggel minden magyarért, kinyilvánítjuk az alábbiakat:',
 'Büszkék vagyunk arra, hogy Szent István királyunk ezer évvel ezelőtt szilárd alapokra helyezte a magyar államot, és hazánkat a keresztény Európa részévé tette.']

In [9]:
lines = sc.textFile('alkotmany.txt')\
.map(lambda l: l.split(" "))

lines.take(2)

[['Mi,',
  'a',
  'magyar',
  'nemzet',
  'tagjai,',
  'az',
  'új',
  'évezred',
  'kezdetén,',
  'felelősséggel',
  'minden',
  'magyarért,',
  'kinyilvánítjuk',
  'az',
  'alábbiakat:'],
 ['Büszkék',
  'vagyunk',
  'arra,',
  'hogy',
  'Szent',
  'István',
  'királyunk',
  'ezer',
  'évvel',
  'ezelőtt',
  'szilárd',
  'alapokra',
  'helyezte',
  'a',
  'magyar',
  'államot,',
  'és',
  'hazánkat',
  'a',
  'keresztény',
  'Európa',
  'részévé',
  'tette.']]

In [10]:
lines = sc.textFile('alkotmany.txt')\
.flatMap(lambda l: l.split(" "))\
.sortBy(lambda word: word)

lines.take(10)

['1944.',
 '1949.',
 '1956-os',
 '1990.',
 'Alaptörvényünk',
 'Becsüljük',
 'Bízunk',
 'Büszkék',
 'Büszkék',
 'Büszkék']

Kulcs-érték párok készítése

Műveletek kulcs-érték párokon: `reduceByKey`, `groupByKey`, `countByKey`

In [11]:
lines = sc.textFile('alkotmany.txt')\
.flatMap(lambda l: l.split(" "))\
.map(lambda w: (w.lower()[0], w.lower()))

lines.take(10)

[('m', 'mi,'),
 ('a', 'a'),
 ('m', 'magyar'),
 ('n', 'nemzet'),
 ('t', 'tagjai,'),
 ('a', 'az'),
 ('ú', 'új'),
 ('é', 'évezred'),
 ('k', 'kezdetén,'),
 ('f', 'felelősséggel')]

Feladat: WordCount

Kulcs alapján aggregáló függvények: `reduceByKey`, `aggregateByKey`, `groupByKey`, `sortByKey`


In [12]:
lines = sc.textFile('alkotmany.txt')\
.flatMap(lambda l: l.split(" "))\
.map(lambda l: l.replace(':',"").replace(",","").replace(".", "").replace("!",""))\
.map(lambda w: (w.lower(), 1))\
.reduceByKey(lambda a, b: a + b)\
.sortBy(lambda x: x[1], False)

lines.take(5)

[('a', 41), ('és', 24), ('az', 19), ('hogy', 17), ('valljuk', 9)]